In [1]:
!git clone https://github.com/brianhie/viral-mutation.git
%cd viral-mutation
!mkdir data
!mkdir data/aav

Cloning into 'viral-mutation'...
remote: Enumerating objects: 1111, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 1111 (delta 133), reused 117 (delta 66), pack-reused 890
Receiving objects: 100% (1111/1111), 177.41 MiB | 11.59 MiB/s, done.
Resolving deltas: 100% (741/741), done.
/content/viral-mutation


In [2]:
!pip install -r  requirements.txt

     |████████████████████████████████| 2.3 MB 4.1 MB/s 
     |████████████████████████████████| 136 kB 59.0 MB/s 
     |████████████████████████████████| 363 kB 42.5 MB/s 
     |████████████████████████████████| 377 kB 38.0 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 13.1 MB 29.4 MB/s 
     |████████████████████████████████| 1.5 MB 45.4 MB/s 
     |████████████████████████████████| 20.3 MB 47.5 MB/s 
     |████████████████████████████████| 10.4 MB 7.4 kB/s 
     |████████████████████████████████| 23.7 MB 296 kB/s 
     |████████████████████████████████| 3.2 MB 35.2 MB/s 
     |████████████████████████████████| 6.5 MB 36.3 MB/s 
     |████████████████████████████████| 25.2 MB 9.7 MB/s 
     |████████████████████████████████| 6.7 MB 26.2 MB/s 
     |████████████████████████████████| 208 kB 55.0 MB/s 
     |████████████████████████████████| 394.5 MB 33 kB/s 
     |████████████████████████████████| 981 kB 60.1 MB/s 
     |█████████

In [3]:
!pip install anndata scanpy biopython

  Using cached anndata-0.8.0-py3-none-any.whl (96 kB)
     |████████████████████████████████| 2.0 MB 4.1 MB/s 
     |████████████████████████████████| 2.3 MB 42.6 MB/s 
     |████████████████████████████████| 11.2 MB 54.0 MB/s 
  Using cached umap-learn-0.5.3.tar.gz (88 kB)
     |████████████████████████████████| 900 kB 43.1 MB/s 
     |████████████████████████████████| 1.1 MB 56.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=75ea11bc64490c2c3584e5c79d23be38fc8d433e135c54b59ccce803e2ea36f9
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=29536beeecb33ad725b3c043ec9b120d6bde3c0165014522af7c14aa0b22c0d8
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80d72741e400345b5a5b50ec3d929aca581bf45e0225d5c50
  Created wheel for s

In [4]:
%cd bin

from mutation import *

np.random.seed(1)
random.seed(1)

/content/viral-mutation/bin


In [5]:
if 1:
    AAs = [
        'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H',
        'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W',
        'Y', 'V', 'X', 'Z', 'J', 'U', 'B', 'Z'
    ]
    vocabulary = { aa: idx + 1 for idx, aa in enumerate(sorted(AAs)) }

In [6]:
def load_meta(meta_fnames):
    metas = {}
    for fname in meta_fnames:
        with open(fname) as f:
            for line in f:
                fields = line.rstrip().split('\t')

                accession, host, genus, family, protein = (
                    fields[0], fields[19], fields[6], fields[7], fields[15]
                )

                metas[accession] = {
                    'host': host,
                    'genus': genus,
                    'family': family,
                    'protein': protein,
                }
    return metas


def process(fnames, meta_fnames):
    metas = load_meta(meta_fnames)

    seqs = {}
    for fname in fnames:
        for record in SeqIO.parse(fname, 'fasta'):
            if record.seq not in seqs:
                seqs[record.seq] = []
            accession = record.description.split(' ')[0]

            if '|' in accession:
              accession = accession.split('|')[1]
            seqs[record.seq].append(metas[accession])
    return seqs

In [7]:
fnames = [ '/content/viral-mutation/data/aav/piccovirales.fa' ]
meta_fnames = [ '/content/viral-mutation/data/aav/piccovirales_metadata.txt' ]

seqs = process(fnames, meta_fnames)
seq_len = max([ len(seq) for seq in seqs ]) + 2
vocab_size = len(AAs) + 2

In [8]:
from language_model import BiLSTMLanguageModel

model = BiLSTMLanguageModel(
    seq_len,
    vocab_size,
    embedding_dim=20,
    hidden_dim=512,
    n_hidden=2,
    n_epochs=4,
    batch_size=200,
    inference_batch_size=200,
    cache_dir='./{}'.format('aav_like'),
    seed=1,
    verbose=True,
    )

In [9]:
#model.model_.load_weights('/content/viral-mutation/models/flu.hdf5')
model.model_.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1191)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1191)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1191, 20)     580         ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lstm (LSTM)                    (None, 1191, 512)    1091584     ['embedding[0][0]',          

In [10]:
def featurize_seqs(seqs, vocabulary):
    start_int = len(vocabulary) + 1
    end_int = len(vocabulary) + 2
    sorted_seqs = sorted(seqs.keys())
    X = np.concatenate([
        np.array([ start_int ] + [
            vocabulary[word] for word in seq
        ] + [ end_int ]) for seq in sorted_seqs
    ]).reshape(-1, 1)
    lens = np.array([ len(seq) + 2 for seq in sorted_seqs ])
    assert(sum(lens) == X.shape[0])
    return X, lens

X, lengths = featurize_seqs(seqs, vocabulary)


In [ ]:
model.fit(X, lengths)

In [ ]:
report_performance('bilstm', model, vocabulary, train_seqs, val_seqs)